<a href="https://colab.research.google.com/github/fasthill/ML-DL-study-alone/blob/main/5-1%20%EA%B2%B0%EC%A0%95%20%ED%8A%B8%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analyse from LightGBM

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce

In [2]:
from xgboost import XGBClassifier
import xgboost
from lightgbm import LGBMClassifier
import lightgbm

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
from sklearn.linear_model import LogisticRegression, SGDClassifier, SGDRegressor
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
# confusion matrix to list 변환
def matrix_to_list(confu_matrix):
    m_list = []
    tn = confu_matrix[0,0]
    fp = confu_matrix[0,1]
    fn = confu_matrix[1,0]
    tp = confu_matrix[1,1]
    m_list.extend([tn, fp, fn, tp])
    return m_list

In [5]:
def predict_p(test_target, y_predict_list): 
    ps = precision_score(test_target, y_predict_list)
    rs = recall_score(test_target, y_predict_list)
    fs = f1_score(test_target, y_predict_list)
    roc = roc_auc_score(test_target, y_predict_list)
#     cm = matrix_to_list(confusion_matrix(test_target, y_predict_list))
    collect_list = [ps, rs, fs, roc]
#     collect_list.extend(cm)
    return collect_list

In [6]:
code = {'005930' : ['삼성전자', 'sec'], '373220' : ['LG에너지솔루션', 'lgenergy'], 
        '000660' : ['SK하이닉스', 'skhinix'], '207940' : ['삼성바이오로직스', 'ssbio'],
        '006400' : ['삼성SDI', 'sdi'], '051910' : ['LG화학', 'lgchemical'],
        '005935' : ['삼성전자우', 'secpre'], '005380' : ['현대차', 'hyunmotor'],
        '035420' : ['NAVER', 'naver'], '000270' : ['기아','kia'],
        '035720' : ['카카오', 'kakao'], '005490' : ['POSCO홀딩스', 'poscoholding'],
        '105560' : ['KB금융', 'kbbank'], '028260' : ['삼성물산', 'sscnt'],
        '068270' : ['셀트리온', 'celltrion'], '012330' : ['현대모비스', 'mobis'],
        '055550' : ['신한지주', 'shgroup'], '066570' : ['LG전자', 'lgelec'],
        '003670' : ['포스코케미칼', 'poscochemical'], '096770' : ['SK이노베이션', 'skinnovation'],
        '033780' : ['KT&G', 'ktng']}

code = {'005930' : ['삼성전자', 'sec']}

In [6]:
fname = f'df_naver_sel.pkl'
directory_for_ml = '../data/data_for_ml/'
f_name = directory_for_ml + fname
df = pd.read_pickle(f_name) 

# train, val,: 8, test: 2
split_ratio = 0.8
split_n = int(len(df)*split_ratio)

data = df.iloc[:split_n, :-5]
target = df.iloc[:split_n, -4]
test_input = df.iloc[split_n:, :-5]
test_target = df.iloc[split_n:, -4]

train_input, val_input, train_target, val_target = \
     train_test_split(data, target, random_state=42, test_size=0.2, stratify=target)

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
val_scaled = ss.transform(val_input)
test_scaled = ss.transform(test_input)

In [7]:
lgbm = None
lgbm = lightgbm.LGBMClassifier(random_state=42,
                               learning_rate =0.22, # default = 0.1
                               num_iterations=500, # default=100
                               max_depth=5, # <= 0 means no limit, default=-1
                               bagging_fraction=0.9, # 0.0 < bagging_fraction <= 1.0, default=1
                               feature_fraction=0.8, # 0.0 < feature_fraction <= 1.0, default=1
                               objective= 'binary', 
#                                num_threads = 0, # == nthread=4, # Gpu 수, default=0, 자동 검색후 적용
                               max_delta_step = 0.5, # best value found, default = 0
#                             scale_pos_weight=40, # class imbalance 경감, scale_pos_weight > 0.0, default=1.0
                              ) 
lgbm.fit(train_scaled, train_target, eval_metric = 'logloss')
y_pred = lgbm.predict(val_scaled)
# val_error = mean_squared_error(val_target, y_pred) # 책에 없음
# print("Validation MSE:", val_error)           # 책에 없음

train_score_lgbm = lgbm.score(train_scaled, train_target)
val_score_lgbm = lgbm.score(val_scaled, val_target)
test_score_lgbm = lgbm.score(test_scaled, test_target)
ps, rs, fs, roc = predict_p(test_target, lgbm.predict(test_scaled))
cm = confusion_matrix(test_target, lgbm.predict(test_scaled))

print("train accuracy: {:.4f}, val accuracy {:.4f}, test accuracy {:.4f}".
      format(train_score_lgbm, val_score_lgbm, test_score_lgbm))
print("precision : {:.4f}, recall : {:.4f}, f1score : {:.4f}, roc : {:.4f}".
     format (ps, rs, fs, roc))
print(cm)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
train accuracy: 1.0000, val accuracy 0.8205, test accuracy 0.7083
precision : 0.6111, recall : 0.6111, f1score : 0.6111, roc : 0.6889
[[23  7]
 [ 7 11]]


c:\users\user\appdata\local\programs\python\python39\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [19]:
params = {
    "boosting_type" : ['gbdt'],
    "max_depth": [None, 2, 3, 4],
    "num_leaves": [3, 4, 5],
    "learning_rate": [0.002, 0.003, 0.004, 0.005],
    "objective": ['binary'],
    "metric": ['loss'],
    "colsample_bytree": [0.38, 0.39, 0.41, 0.42],
    "subsample": [0.003, 0.004, 0.005, 0.01],
    "n_estimators": [3, 4, 5, 6, 7, 8],
#     "num_iterations": [100, 200, 300, 400, 500],
#     "num_class": [1]
#     "metric" : "rmse",
#     "bagging_frequency" : 5,
#     "bagging_seed" : 2018,
#     "verbosity" : -1,

#     # Selected rounded-off params
#     'bagging_fraction': 0.7,
#     'feature_fraction': 0.1,
#     'lambda_l1': 1,
#     'lambda_l2': 0,
#     'max_depth': 9,
#     'min_child_weight': 5,
#     'min_split_gain': 0,
#     'num_leaves': 24
}
lgbm = None
lgbmgs = None

lgbm = lightgbm.LGBMClassifier( random_state=42, 
                               learning_rate =0.22, # default = 0.1
                               num_iterations=500, # default=100
                               max_depth=5, # <= 0 means no limit, default=-1
#                                bagging_fraction=0.9, # 0.0 < bagging_fraction <= 1.0, default=1
#                                feature_fraction=0.8, # 0.0 < feature_fraction <= 1.0, default=1
                               objective= 'binary', 
#                                num_threads = 0, # == nthread=4, # Gpu 수, default=0, 자동 검색후 적용
                               max_delta_step = 0.5, # best value found, default = 0
#                                scale_pos_weight=40, # class imbalance 경감, scale_pos_weight > 0.0, default=1.0
                               eval_metric = 'logloss'
                                )

lgbmgs = GridSearchCV(estimator = lgbm,
                      param_grid = params,
                      cv = 5, # StratifiedKFold us default for binary or multiclass
#                       scoring = 'precision', 
                      scoring = 'accuracy', 
                      verbose = 1,
                      n_jobs=-1, # 자동 검색 적용
                      )

lgbmgs.fit(train_scaled, train_target)
    
print("Best Estimator: {}".format(lgbmgs.best_estimator_)) 
print("Best Parameters: {}".format(lgbmgs.best_params_))  # 최적 파라미터.
print('Best Score: {}, Best Index: {}'.format(lgbmgs.best_score_ , lgbmgs.best_index_))  # 교차검증된 점수를 보여줌.

model = lgbmgs.best_estimator_  # 최적의 파라미터로 모델 생성
y_pred = model.predict(test_scaled)

train_score = model.score(train_scaled, train_target)
val_score = model.score(val_scaled, val_target)
test_score = model.score(test_scaled, test_target)
ps, rs, fs, roc = predict_p(test_target, model.predict(test_scaled))
cm = confusion_matrix(test_target, model.predict(test_scaled))

print("train accuracy: {:.4f}, val accuracy {:.4f}, test accuracy {:.4f}".
      format(train_score, val_score, test_score))
print("precision : {:.4f}, recall : {:.4f}, f1score : {:.4f}, roc : {:.4f}".
     format (ps, rs, fs, roc))
print(cm)

Fitting 5 folds for each of 4608 candidates, totalling 23040 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.38, eval_metric='logloss',
               learning_rate=0.005, max_delta_step=0.5, max_depth=None,
               metric='loss', n_estimators=3, num_iterations=500, num_leaves=3,
               objective='binary', random_state=42, subsample=0.003)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.38, 'learning_rate': 0.005, 'max_depth': None, 'metric': 'loss', 'n_estimators': 3, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.003}
Best Score: 0.8311827956989248, Best Index: 864
train accuracy: 0.8758, val accuracy 0.8974, test accuracy 0.7708
precision : 0.8182, recall : 0.5000, f1score : 0.6207, roc : 0.7167
[[28  2]
 [ 9  9]]


c:\users\user\appdata\local\programs\python\python39\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [12]:
model.feature_importances_

array([ 36,   5,   5,   4,  11,   5,   2,   0,   0,   6,   0,   7,   0,
         2,  31,  18,  11,   8,  32,  29,   7,   4,   3,   0, 162, 179,
        32,  13,   0,   1,  21, 175, 187,   0,   0])

In [13]:
df_sel = pd.DataFrame(model.feature_importances_, index=data.columns, columns=['importance']).sort_values(by='importance', ascending=False)
df_sel

,importance
low,187
kospi_cr,179
high,175
kosdaq_cr,162
retail,36
krw_cr,32
bond_kor_10_cr,32
ixic_cr,31
bond_kor_2_cr,29
open,21


In [10]:
model.booster_.feature_importance(importance_type='gain')

array([2.01853705e+02, 2.09116001e+01, 2.24134402e+01, 1.99464099e+01,
       4.68392494e+01, 1.82933300e+01, 7.76625991e+00, 0.00000000e+00,
       0.00000000e+00, 2.44078705e+01, 0.00000000e+00, 3.46162000e+01,
       0.00000000e+00, 8.73318966e-03, 1.55219209e+01, 3.39758409e+01,
       8.69237512e+00, 8.67263984e+00, 1.03369987e+02, 6.71910275e+01,
       3.29085801e+01, 1.01570300e+01, 1.17823501e+01, 0.00000000e+00,
       1.65933056e+03, 3.50580067e+03, 3.56548126e+01, 5.08107218e+00,
       0.00000000e+00, 6.88262987e+00, 7.59577905e+01, 2.18379700e+03,
       5.84932801e+03, 0.00000000e+00, 0.00000000e+00])

In [14]:
df_sel = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), 
                      index=data.columns, columns=['importance']).sort_values(by='importance', 
                                                                              ascending=False)
df_sel

,importance
low,5849.328011
kospi_cr,3505.800665
high,2183.797003
kosdaq_cr,1659.330565
retail,201.853705
bond_kor_10_cr,103.369987
open,75.957790
bond_kor_2_cr,67.191028
invtrust,46.839249
krw_cr,35.654813


In [18]:
df_sel = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), 
                      index=data.columns, columns=['importance']).sort_values(by='importance', 
                                                                              ascending=False)
df_sel

,importance
low,4807.865055
kospi_cr,3317.561685
high,1831.948064
kosdaq_cr,1494.045491
retail,182.058875
bond_kor_2_cr,118.091723
bond_kor_10_cr,89.554625
open,45.789817
financeetc,42.706240
foreigneretc,41.495180


In [11]:
model.booster_.feature_importance(importance_type='split')
# 큰 특징을 가지는 feature는 tree상위레벨에서 적게 사용됨.

array([ 36,   5,   5,   4,  11,   5,   2,   0,   0,   6,   0,   7,   0,
         2,  31,  18,  11,   8,  32,  29,   7,   4,   3,   0, 162, 179,
        32,  13,   0,   1,  21, 175, 187,   0,   0])

In [ ]:
model.booster_.feature_importance(importance_type='gain')

In [53]:
for i in ['mean_test_score', 'std_test_score']:
        print(i," : ", gsearch1.cv_results_[i])

mean_test_score  :  [0.5        0.91806667        nan 0.5        0.90966667        nan
 0.5        0.91546667        nan 0.5        0.91293333        nan
 0.5        0.90963333        nan 0.5        0.9145            nan
 0.5        0.90633333        nan 0.5        0.9128            nan
 0.5        0.9112            nan 0.5        0.91456667        nan
 0.5        0.9088            nan 0.5        0.91283333        nan
 0.5        0.90803333        nan 0.5        0.91466667        nan
 0.5        0.90866667        nan 0.5        0.90796667        nan
 0.5        0.91706667        nan 0.5        0.92043333        nan
 0.5        0.90883333        nan 0.5        0.90966667        nan
 0.5        0.91113333        nan 0.5        0.89726667        nan
 0.5        0.89386667        nan 0.5        0.9096            nan
 0.5        0.90566667        nan 0.5        0.90386667        nan
 0.5        0.90383333        nan 0.5        0.91293333        nan
 0.5        0.90873333        nan 0.5     

In [36]:
gsearch1.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_min_child_weight', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [52]:
# use best fit model using .best_estimator

best_xgb = gsearch1.best_estimator_
y_pred = best_xgb.predict(val_scaled)
# val_error = mean_squared_error(val_target, y_pred) # 책에 없음
# print("Validation MSE:", val_error)           # 책에 없음

train_score_xgb = best_xgb.score(train_scaled, train_target)
val_score_xgb = best_xgb.score(val_scaled, val_target)
test_score_xgb = best_xgb.score(test_scaled, test_target)
ps, rs, fs, roc = predict_p(test_target, best_xgb.predict(test_scaled))
cm = confusion_matrix(test_target, best_xgb.predict(test_scaled))

print("train accuracy: {:.4f}, val accuracy {:.4f}, test accuracy {:.4f}".
      format(train_score_xgb, val_score_xgb, test_score_xgb))
print("precision : {:.4f}, recall : {:.4f}, f1score : {:.4f}, roc : {:.4f}".
     format (ps, rs, fs, roc))
print(cm)

train accuracy: 0.9884, val accuracy 0.8140, test accuracy 0.8148
precision : 0.6667, recall : 0.8235, f1score : 0.7368, roc : 0.8172
[[30  7]
 [ 3 14]]


Naver
Fitting 5 folds for each of 1296 candidates, totalling 6480 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.41, eval_metric='logloss',
               learning_rate=0.004, max_delta_step=0.5, max_depth=2,
               metric='loss', n_estimators=5, num_iterations=500, num_leaves=3,
               objective='binary', random_state=42, subsample=0.005)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.41, 'learning_rate': 0.004, 'max_depth': 2, 'metric': 'loss', 'n_estimators': 5, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.005}
Best Score: 0.9444444444444444, Best Index: 0
train accuracy: 0.8758, val accuracy 0.8974, test accuracy 0.7917
precision : 0.9000, recall : 0.5000, f1score : 0.6429, roc : 0.7333
[[29  1]
 [ 9  9]]

Fitting 5 folds for each of 3456 candidates, totalling 17280 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.38, eval_metric='logloss',
               learning_rate=0.003, max_delta_step=0.5, max_depth=2,
               metric='loss', n_estimators=3, num_iterations=500, num_leaves=3,
               objective='binary', random_state=42, subsample=0.003)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.38, 'learning_rate': 0.003, 'max_depth': 2, 'metric': 'loss', 'n_estimators': 3, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.003}
Best Score: 0.9777777777777779, Best Index: 216
train accuracy: 0.8170, val accuracy 0.8462, test accuracy 0.7292
precision : 0.8571, recall : 0.3333, f1score : 0.4800, roc : 0.6500
[[29  1]
 [12  6]]

#### scoring을 precision에서 default(accuracy)로 변경한 결과
Fitting 5 folds for each of 3456 candidates, totalling 17280 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.38, eval_metric='logloss',
               learning_rate=0.005, max_delta_step=0.5, max_depth=2,
               metric='loss', n_estimators=3, num_iterations=500, num_leaves=3,
               objective='binary', random_state=42, subsample=0.003)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.38, 'learning_rate': 0.005, 'max_depth': 2, 'metric': 'loss', 'n_estimators': 3, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.003}
Best Score: 0.8311827956989248, Best Index: 648
train accuracy: 0.8758, val accuracy 0.8974, test accuracy 0.7708
precision : 0.8182, recall : 0.5000, f1score : 0.6207, roc : 0.7167
[[28  2]
 [ 9  9]]

#### max_depth = None 삽입결과

Fitting 5 folds for each of 4608 candidates, totalling 23040 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(colsample_bytree=0.38, eval_metric='logloss',
               learning_rate=0.005, max_delta_step=0.5, max_depth=None,
               metric='loss', n_estimators=3, num_iterations=500, num_leaves=3,
               objective='binary', random_state=42, subsample=0.003)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.38, 'learning_rate': 0.005, 'max_depth': None, 'metric': 'loss', 'n_estimators': 3, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.003}
Best Score: 0.8311827956989248, Best Index: 864
train accuracy: 0.8758, val accuracy 0.8974, test accuracy 0.7708
precision : 0.8182, recall : 0.5000, f1score : 0.6207, roc : 0.7167
[[28  2]
 [ 9  9]]